In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
import json
import powerlaw
import fiona
#import geopandas as gdp

ImportError: dlopen(/opt/homebrew/lib/python3.9/site-packages/fiona/ogrext.cpython-39-darwin.so, 0x0002): Library not loaded: /opt/homebrew/opt/gdal/lib/libgdal.31.dylib
  Referenced from: <4C8E2433-AA31-3D2F-BCDC-F4FFF503B2A5> /opt/homebrew/lib/python3.9/site-packages/fiona/ogrext.cpython-39-darwin.so
  Reason: tried: '/opt/homebrew/opt/gdal/lib/libgdal.31.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/gdal/lib/libgdal.31.dylib' (no such file), '/opt/homebrew/opt/gdal/lib/libgdal.31.dylib' (no such file), '/usr/local/lib/libgdal.31.dylib' (no such file), '/usr/lib/libgdal.31.dylib' (no such file, not in dyld cache), '/opt/homebrew/Cellar/gdal/3.6.4_7/lib/libgdal.31.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/Cellar/gdal/3.6.4_7/lib/libgdal.31.dylib' (no such file), '/opt/homebrew/Cellar/gdal/3.6.4_7/lib/libgdal.31.dylib' (no such file), '/usr/local/lib/libgdal.31.dylib' (no such file), '/usr/lib/libgdal.31.dylib' (no such fi

In [ ]:
import scipy

In [ ]:
%matplotlib inline

In [ ]:
geo_df = gdp.read_file('geo_data/Parishes_and_Non_Civil_Parished_Areas_(December_2021)_EW_BSC.shp')
geo_df.head()

In [ ]:
oa_to_parish_lookup = pd.read_csv('data/OAs_(2021)_to_Civil_Parish_and_Non-Civil_Parished_Areas_to_LTLA_to_UTLA_to_Region_to_Country_(May_2022)_Lookup_in_England_and_Wales.csv')
oa_to_parish_lookup.head()

In [ ]:
oa_to_parish_lookup = oa_to_parish_lookup[[
    'OA21CD',
    'PAR22CD',
    'PAR22NM',
    'RGN22NM'
]]

oa_to_parish_lookup.head()

In [ ]:
oa_populations = pd.read_csv('data/census2021-ts001-oa.csv')
oa_populations.head()

In [ ]:
oa_total_frame = oa_to_parish_lookup.merge(
    oa_populations[['geography code', 'Residence type: Total; measures: Value']], 
    how='inner', 
    left_on='OA21CD', 
    right_on='geography code')

oa_total_frame.head()
                    

In [ ]:
oa_total_frame = oa_total_frame.rename(columns={'Residence type: Total; measures: Value': 'population'}, errors='raise')
oa_total_frame = oa_total_frame[[
    'OA21CD',
    'PAR22CD',
    'PAR22NM',
    'RGN22NM',
    'population'
]]

oa_total_frame.head()

In [ ]:
size_frame = pd.read_csv('geo_data/Output_Areas_Dec_2021_Boundaries_Full_Clipped_EW_BFC_2022_-340120420164038365.csv')
size_frame.head()

In [ ]:
oa_total_frame = oa_total_frame.merge(
    size_frame[['OA21CD', 'Shape__Area']],
    how='inner',
    on='OA21CD'
)
oa_total_frame = oa_total_frame.rename(
    columns={'Shape__Area': 'area'}, 
    errors='raise'
)

In [ ]:
oa_total_frame.head()

In [ ]:
import glob
to_include = list(set(glob.glob('output_data/all_data_final_updated_*.json')) - set(glob.glob('output_data/all_data_final_updated_errors*.json')))
region_frames = [pd.read_json(i) for i in to_include]
big_df = pd.concat(region_frames)
big_df.shape


In [ ]:
oa_total_frame = oa_total_frame.merge(
    big_df[['OA21CD', 'amenities']],
    how='left',
    on='OA21CD'
)
oa_total_frame.head()

In [ ]:
oa_total_frame.tail()

In [ ]:
## drop Wales
drop_wales = oa_total_frame[oa_total_frame.RGN22NM != 'Wales']
drop_wales.shape

In [ ]:
drop_wales[pd.isnull(drop_wales.amenities)].groupby(['RGN22NM']).size()

In [ ]:
oa_to_parish_lookup.groupby(['RGN22NM']).size()

In [ ]:
parish_frame = drop_wales[['PAR22CD', 'PAR22NM', 'RGN22NM', 'population', 'area']].groupby(['PAR22CD', 'PAR22NM', 'RGN22NM']).agg('sum').reset_index()
parish_frame.head()

## Adding on Rural/Urban classification

In [ ]:
rural_urban_lookup = pd.read_csv('data/RUC11_OA11_EW.csv')
rural_urban_lookup.head()

In [ ]:
oa11_oa21_lookup = pd.read_csv('data/OA_(2011)_to_OA_(2021)_to_Local_Authority_District_(2022)_for_England_and_Wales_Lookup_(Version_2).csv')
oa11_oa21_lookup.head()

In [ ]:
rural_urban_lookup = rural_urban_lookup[['OA11CD', 'RUC11CD', 'RUC11']].merge(
    oa11_oa21_lookup[['OA11CD', 'OA21CD']],
    how='left',
    on='OA11CD'
)
rural_urban_lookup.head()

In [ ]:
with_rural = drop_wales.merge(
    rural_urban_lookup[[
        'OA21CD',
        'RUC11CD',
        'RUC11'
    ]],
    how='left',
    on='OA21CD'
)

with_rural.head()

In [ ]:
with_rural.groupby(['RUC11'])[[ 'area']].describe()

In [ ]:
with_rural[['RUC11', 'RUC11CD']].drop_duplicates()

In [ ]:
official_rural_list = [
    'D1', 'E1', 'F1', 'D2', 'E2', 'F2'
]

with_rural.head()

In [ ]:
with_rural['is_rural'] = with_rural.RUC11CD.isin(official_rural_list)
with_rural.head()

In [ ]:
with_rural.groupby(['is_rural']).describe()

In [ ]:
agg_stats = with_rural.groupby(['is_rural']).sum().reset_index()
agg_stats

In [ ]:
agg_stats['pct_population'] = 100*agg_stats.population/agg_stats.population.sum()
agg_stats['pct_area'] = 100*agg_stats.area/agg_stats.area.sum()
agg_stats

## Facilities by urban/rural

In [ ]:
import collections

def extract_amenity_list(value):
    return collections.Counter([entry['amenity'] for entry in value])

In [ ]:
with_rural['amenity_list'] = with_rural.amenities.apply(extract_amenity_list)
with_rural.head()

In [ ]:
amenity_counters = with_rural.groupby(['is_rural']).amenity_list.sum(collections.Counter()).reset_index()
amenity_counters

In [ ]:
rural_counter = amenity_counters[amenity_counters.is_rural].amenity_list.values[0]
urban_counter = amenity_counters[~amenity_counters.is_rural].amenity_list.values[0]

In [ ]:
urban_counter.most_common(10)

In [ ]:
rural_counter.most_common(10)

In [ ]:
with_rural.groupby('is_rural').size()

In [ ]:
normalised_rural = collections.Counter()
total_rural = sum(rural_counter.values())
for item, count in rural_counter.items():
    normalised_rural[item] = 100*count/total_rural
    
normalised_urban = collections.Counter()
total_urban = sum(urban_counter.values())
for item, count in urban_counter.items():
    normalised_urban[item] = 100*count/total_urban

In [ ]:
normalised_urban.most_common(10)

In [ ]:
normalised_rural.most_common(10)

In [ ]:
rural_population=with_rural[with_rural.is_rural].population.sum()
rural_oas = with_rural[with_rural.is_rural].shape[0]
urban_population=with_rural[~with_rural.is_rural].population.sum()
urban_oas = with_rural[~with_rural.is_rural].shape[0]

rural_frame = pd.DataFrame(rural_counter.items(), columns=['amenity', 'number'])
rural_frame['pct_items'] = 100*rural_frame.number / rural_frame.number.sum()
rural_frame['num_per_person'] = rural_frame.number / rural_population
rural_frame['num_per_oa'] = rural_frame.number / rural_oas
urban_frame = pd.DataFrame(urban_counter.items(), columns=['amenity', 'number'])
urban_frame['pct_items'] = 100*urban_frame.number / urban_frame.number.sum()
urban_frame['num_per_person'] = urban_frame.number / urban_population
urban_frame['num_per_oa'] = urban_frame.number / urban_oas
urban_frame.head()

In [ ]:
mergey = urban_frame.merge(rural_frame, how='left', on='amenity', suffixes=('_urban', '_rural'))
mergey.head()

In [ ]:
mergey['pct_diff'] = mergey.pct_items_urban - mergey.pct_items_rural
mergey['diff_per_person'] = mergey.num_per_person_urban - mergey.num_per_person_rural
mergey['diff_per_oa'] = mergey.num_per_oa_urban - mergey.num_per_oa_rural
mergey.head()

In [ ]:
mergey.sort_values(by='pct_diff').head(10)

In [ ]:
mergey.sort_values(by='pct_diff', ascending=False).head(10)

In [ ]:
mergey.sort_values(by='diff_per_person').head(10)

In [ ]:
mergey.sort_values(by='diff_per_person', ascending=False).head(10)

## Classifying parishes

In [ ]:
with_rural.head()

In [ ]:
parish_frame.head()

In [ ]:
parish_amenities = with_rural.groupby(['PAR22CD', 'PAR22NM', 'RGN22NM']).amenity_list.sum(collections.Counter()).reset_index()
parish_amenities.head()

In [ ]:
parish_data = parish_frame.merge(parish_amenities, how='inner', on=['PAR22CD', 'PAR22NM', 'RGN22NM'])
parish_data.head()

In [ ]:
parish_data.shape

In [ ]:
classification = with_rural.groupby(['PAR22CD']).is_rural.apply(lambda x: x.sum() / x.size).reset_index()
classification.head()

In [ ]:
def classify_parish(x):
    if x == 0:
        return 'Urban'
    elif x == 1:
        return 'Rural'
    else:
        return 'Mixed'
classification['rural_label'] = classification.is_rural.apply(classify_parish)
classification.head()

In [ ]:
classification.groupby(['rural_label']).size()

In [ ]:
parish_data = parish_data.merge(classification[['PAR22CD', 'rural_label']], how='inner', on='PAR22CD')
parish_data.head()

In [ ]:
parish_data.groupby(['rural_label']).describe()

In [ ]:
parish_data.head()

In [ ]:
amenity_list = ['pub', 'school', 'place_of_worship']

In [ ]:
amenity_counts = [
    [i.get(amenity,0) for i in parish_data.amenity_list]
    for amenity in amenity_list
]

for amenity in amenity_list:
    parish_data[amenity] = [i.get(amenity, 0) for i in parish_data.amenity_list]
    
parish_data.head()

## Plotting attempts

In [ ]:
import geopandas as gdp
geo_df = gdp.read_file('geo_data/Parishes_and_Non_Civil_Parished_Areas_(December_2021)_EW_BSC.shp')
geo_df = geo_df.to_crs('EPSG:4326')
geo_df.head()

In [ ]:
import statsmodels.api as sm
from statsmodels.genmod.families import NegativeBinomial
from statsmodels.discrete.count_model import ZeroInflatedPoisson
# Fit the Negative Binomial model
negbin_model = sm.GLM(amenity_counts[0], sm.add_constant(np.log(parish_data.population)),
                      family=sm.families.NegativeBinomial()).fit()

# Fit the Zero-Inflated Poisson model
zip_model = ZeroInflatedPoisson(np.array(amenity_counts[0]), sm.add_constant(np.log(parish_data.population))).fit()



# Compare model performance using AIC or BIC
print(negbin_model.summary())
print(zip_model.summary())

In [ ]:
print(negbin_model.aic)
print(zip_model.aic)

In [ ]:
# Fit the negative binomial regression model
model = sm.GLM(
    np.log(parish_data[parish_data.rural_label == 'Rural'].pub + 1), 
    sm.add_constant(
        np.log(parish_data[parish_data.rural_label=='Rural'].population)
    ),
    family=sm.families.NegativeBinomial()
).fit()

# Print the model summary
print(model.summary())

In [ ]:
residuals = model.resid_response
fitted_values = model.fittedvalues
plt.scatter(fitted_values, residuals)
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Fitted')
plt.show()

# Quantile-Quantile plot of the residuals
sm.qqplot(residuals, line='s')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Sample Quantiles')
plt.title('Q-Q Plot of Residuals')
plt.show()

## Layering on IMD

In [ ]:
imd_data = pd.read_csv('data/Index_of_Multiple_Deprivation_(Dec_2019)_Lookup_in_England.csv')
imd_data.head()

In [ ]:
oa11_lsoa11_lookup = pd.read_csv('data/Output_Area_to_Lower_layer_Super_Output_Area_to_Middle_layer_Super_Output_Area_to_Local_Authority_District_(December_2011)_Lookup_in_England_and_Wales.csv')
oa11_lsoa11_lookup.head()


In [ ]:
oa11_oa21_lookup.head()

In [ ]:
imd_frame = oa11_oa21_lookup[['OA11CD', 'OA21CD']].merge(
    oa11_lsoa11_lookup[['OA11CD', 'LSOA11CD']], how='inner', on='OA11CD'
).merge(
    imd_data[['LSOA11CD', 'IMD19']],
    how='inner',
    on='LSOA11CD'
)

imd_frame.head()

In [ ]:
oa_total_frame.head()

In [ ]:
imd_frame.groupby(['OA21CD']).size().sort_values(ascending=False).head(10)

In [ ]:
raw_oa21_imd_lookup = imd_frame[['OA21CD', 'IMD19']].drop_duplicates()
oa_total_frame = oa_total_frame.merge(raw_oa21_imd_lookup, how='left', on='OA21CD')
oa_total_frame.head()

In [ ]:
parish_data.head()

In [ ]:
def weighted_imd(x,weighting_column):
    imd_total = 0
    weighting_total = 0
    for row in x.iterrows():
        imd_total += row[1].IMD19 * row[1][weighting_column]
        weighting_total += row[1][weighting_column]
    total = imd_total/weighting_total
    return total

parish_imd_frame = oa_total_frame.groupby(['PAR22CD', 'PAR22NM']).apply(lambda x: weighted_imd(x, 'population')).reset_index()
parish_imd_frame.columns = ['PAR22CD', 'PAR22NM', 'population_weighted_imd']
parish_imd_frame.head()


In [ ]:
inty = oa_total_frame.groupby(['PAR22CD', 'PAR22NM']).apply(lambda x: weighted_imd(x, 'area')).reset_index()
inty.columns = ['PAR22CD', 'PAR22NM', 'area_weighted_imd']
inty.head()

In [ ]:
parish_imd_frame = parish_imd_frame.merge(inty, how='inner', on=['PAR22CD', 'PAR22NM'])
parish_imd_frame.head()

In [ ]:
parish_imd_frame[pd.isnull(parish_imd_frame.area_weighted_imd)].head()

In [ ]:
parish_data = parish_data.merge(
    parish_imd_frame[['PAR22CD', 'population_weighted_imd', 'area_weighted_imd']],
    how='inner',
    on='PAR22CD'
)

parish_data.head()

In [ ]:
parish_data[parish_data.PAR22NM == 'Great Ouseburn']

## Powerlaw time

In [ ]:
fit = powerlaw.Fit(parish_data.pub.values, discrete=True)

In [ ]:
fit.power_law.alpha

In [ ]:
fit.power_law.sigma

In [ ]:
fit.distribution_compare('power_law', 'exponential')

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))

plt.hist(np.log(parish_data.pub.values + 1), bins=30)
plt.show()

In [ ]:
len(parish_data.pub.values)

In [ ]:
np.max(parish_data.pub.values)

In [ ]:
from collections import Counter
Counter(parish_data.pub.values).most_common(10)

In [ ]:
data = parish_data.pub.values
plt.hist(data, bins=range(max(data)+2), align='left')
plt.xlabel('Number of Pubs')
plt.ylabel('Frequency')
plt.title('Distribution of Pubs in a Parish')
plt.show()

In [ ]:
from scipy.stats import poisson
mu = np.mean(data)

x = np.arange(0, max(data) + 1)
dist = poisson.pmf(x, mu)
fig, ax = plt.subplots(figsize=(20,12))
plt.hist(data, bins=range(max(data) + 2), align='left', density=True, label='Data')
plt.plot(x, dist, 'r-', label='Poisson fit')
plt.xlabel('Number of Pubs')
plt.ylabel('Probability')
plt.title('Fitted Poisson Distribution')
plt.xlim([0,20])
plt.legend()
plt.show()

In [ ]:
mu

In [ ]:
from scipy.stats import nbinom

fig, ax = plt.subplots(figsize=(20,12))
# Plot the fitted Negative Binomial distribution
x = np.arange(0, max(data) + 1)
plt.hist(data, bins=range(max(data) + 2), align='left', density=True, label='Data')
plt.plot(x, nbinom.pmf(x, 0.056287, 0.01532555), 'r-', label='Negative Binomial fit')
plt.xlabel('Number of Pubs')
plt.ylabel('Probability')
plt.title('Fitted Negative Binomial Distribution')
plt.xlim([0,15])
plt.legend()
plt.show()

In [ ]:
from scipy.special import gammaln, factorial
from scipy.special import psi
from scipy.optimize import fmin_l_bfgs_b as optim

def fit_nbinom(X, initial_params=None):
    infinitesimal = np.finfo(np.float).eps

    def log_likelihood(params, *args):
        r, p = params
        X = args[0]
        N = X.size

        #MLE estimate based on the formula on Wikipedia:
        # http://en.wikipedia.org/wiki/Negative_binomial_distribution#Maximum_likelihood_estimation
        result = np.sum(gammaln(X + r)) \
            - np.sum(np.log(factorial(X))) \
            - N*(gammaln(r)) \
            + N*r*np.log(p) \
            + np.sum(X*np.log(1-(p if p < 1 else 1-infinitesimal)))

        return -result

    def log_likelihood_deriv(params, *args):
        r, p = params
        X = args[0]
        N = X.size

        pderiv = (N*r)/p - np.sum(X)/(1-(p if p < 1 else 1-infinitesimal))
        rderiv = np.sum(psi(X + r)) \
            - N*psi(r) \
            + N*np.log(p)

        return np.array([-rderiv, -pderiv])

    if initial_params is None:
        #reasonable initial values (from fitdistr function in R)
        m = np.mean(X)
        v = np.var(X)
        size = (m**2)/(v-m) if v > m else 10

        #convert mu/size parameterization to prob/size
        p0 = size / ((size+m) if size+m != 0 else 1)
        r0 = size
        initial_params = np.array([r0, p0])

    bounds = [(infinitesimal, None), (infinitesimal, 1)]
    optimres = optim(log_likelihood,
                     x0=initial_params,
                     #fprime=log_likelihood_deriv,
                     args=(X,),
                     approx_grad=1,
                     bounds=bounds)

    params = optimres[0]
    return {'size': params[0], 'prob': params[1]}

In [ ]:
fit_nbinom(data)

In [ ]:
nbinom.pmf(x, 0.01532555, 0.056287)

In [ ]:
from scipy.stats._continuous_distns import _distn_names

In [ ]:
rural_only = parish_data[parish_data.rural_label == 'Rural']
rural_only.shape

In [ ]:
rural_only.head()

In [ ]:
import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from scipy.stats._discrete_distns import _distn_names
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import fit

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')

# Create models from data
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = x[:-1]

    # Best holders
    best_distributions = []

    # Estimate distribution parameters from data
    for ii, distribution in enumerate([d for d in _distn_names if not d in ['levy_stable', 'studentized_range']]):

        print("{:>3} / {:<3}: {}".format( ii+1, len(_distn_names), distribution ))

        distribution = getattr(st, distribution)

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = fit(distribution, data, bounds = {
                    'n': [-1000,1000], 
                    'p': [-1000, 1000], 
                    'mu': [-1000,1000], 
                    'scale': [-1000, 1000],
                    'loc': [-1000,1000],
                    'a': [-1000, 1000],
                    'lambda_': [-1000,1000],
                    'lambda': [-1000,1000],
                    'N': [-1000, 1000],
                    'b': [-1000,1000],
                    'M': [-1000,1000],
                    'mu1': [-1000,1000],
                    'mu2': [-1000,1000],
                    'alpha': [-1000,1000],
#                    'odds': [-10,20]
                })
                score = params.nllf()

                # Separate parts of parameters
                better_params = params.params

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pmf(x,*better_params)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        ax.plot(x, pdf)
#                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                print("About to append")
                best_distributions.append((distribution, better_params, score))
        except Exception:
            pass
            

    
    return sorted(best_distributions, key=lambda x:x[2])

def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters


    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *params) 
    end = dist.ppf(0.99, *params)

    # Build PDF and turn into pandas Series
    x = np.arange(start, end)
    y = dist.pmf(x, *params)
    pdf = pd.Series(y, x)

    return pdf

# Load data from statsmodels datasets
data = pd.Series(parish_data[parish_data.rural_label == 'Rural'].pub.values)

# Plot for comparison
plt.figure(figsize=(12,8))
ax = data.plot(kind='hist', bins=50, density=True, alpha=0.5, color=list(matplotlib.rcParams['axes.prop_cycle'])[1]['color'])

# Save plot limits
dataYLim = ax.get_ylim()

# Find best fit distribution
best_distibutions = best_fit_distribution(data, 200, ax)
best_dist = best_distibutions[0]

# Update plots
ax.set_ylim(dataYLim)
ax.set_title(u'Number of pubs\n All Fitted Distributions')
ax.set_xlabel(u'Number of pubs')
ax.set_ylabel('Frequency')

# Make PDF with best params 
plt.figure(figsize=(20,12))
for i in range(4):
    disty = best_distibutions[i]
    pdf = make_pdf(disty[0], disty[1])
    ax = pdf.plot(lw=2, label='PDF - {0}'.format(disty[0].name), legend=True)
# Display

ax.hist(data, bins=50, label='Data')

#data.plot(kind='hist', bins=50, density=True, alpha=0.5, label='Data', legend=True, ax=ax)

param_names = (best_dist[0].shapes + ', loc, scale').split(', ') if best_dist[0].shapes else ['loc', 'scale']
param_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_names, best_dist[1])])
dist_str = '{}({})'.format(best_dist[0].name, param_str)

ax.set_title(u'Number of pubs with best fit distribution \n' + dist_str)
ax.set_xlabel(u'Number of pubs')
ax.set_ylabel('Frequency')

In [ ]:
np.sum(make_pdf(best_distibutions[0][0], best_distibutions[0][1]))

In [ ]:
best_distibutions

In [ ]:
best_distributions[1]

In [ ]:
a

In [ ]:
start

In [ ]:
end

In [ ]:
y, x = np.histogram(data, bins=200, density=True)
x = x[:-1]
x

In [ ]:
x = np.linspace(start, end, 10000)
y = dist.pmf(x, *params)

In [ ]:
y

In [ ]:
distribution = getattr(st, _distn_names[3])

In [ ]:
params = distribution.fit(data)

In [ ]:
from scipy.stats import fit

In [ ]:
params = fit(distribution, data, bounds={'n': [-10,100], 'p': [-10,100]})

In [ ]:
params

In [ ]:
n = params.params.n
p = params.params.p

In [ ]:
pdf = distribution.pmf(np.arange(50), n=n, p=p)

In [ ]:
pdf

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))
plt.bar(np.arange(50), pdf)
plt.show()

In [ ]:
x

In [ ]:
params = fit(distribution, data, bounds = {'n': [-10,10], 'p': [-10, 10]})

                # Separate parts of parameters
n = params.params.n
p = params.params.p
                
                # Calculate fitted PDF and error with fit in distribution
pdf = distribution.pmf(x, n=n, p=p)

In [ ]:
ax

In [ ]:
pd.Series(pdf, x).plot(ax=ax)

In [ ]:
plt.show()

In [ ]:
from collections import Counter
Counter(data.values).most_common(15)

In [ ]:
best_distibutions

In [ ]:
from scipy import stats
alternative = stats.betabinom.rvs(n=450, a=2.10153167667377, b=698.4475720315137, size=300)
stats.kstest(data.values, alternative)

In [ ]:
stats.nbinom?

In [ ]:
np.max(data)